# Kellis Lab Single-nuclei ATAC-seq Preprocessing Pipeline

---

### Overview

This pipeline preprocesses single-nucleus ATAC-seq (snATAC-seq) data from the Kellis lab for downstream chromatin accessibility QTL (caQTL) analysis and region-specific studies. It processes pseudobulk peak count data across six major brain cell types with flexible workflow options depending on your analysis goals.

**Pipeline Purpose:**
- Transform raw pseudobulk peak counts into analysis-ready formats
- Remove technical confounders and optionally biological covariates
- Generate QTL-ready phenotype files or region-specific datasets

**Supported Cell Types:**
- **Mic** - Microglia
- **Astro** - Astrocytes
- **Oligo** - Oligodendrocytes
- **Exc** - Excitatory neurons
- **Inh** - Inhibitory neurons
- **OPC** - Oligodendrocyte precursor cells

---

### Workflow Structure

This pipeline consists of two main sequential steps, plus a complete pipeline for severe batch effects.

#### Step 1: Pseudobulk QC with batch as covariates

**Option A: Remove Biological Covariates**
- Regresses out demographic variables (msex, age_death, pmi, study)
- **Use when:** You want to identify genetic effects independent of sex/age
- **Model includes:** technical covariates + sequencingBatch + msex + age_death + pmi + study

**Option B: Preserve Biological Covariates**
- Regresses out only non-demographic variables (pmi, study)
- **Use when:** You want to study sex/age effects or preserve biological heterogeneity
- **Model includes:** technical covariates + sequencingBatch + pmi + study (NO msex, age_death)

#### Step 2: Format Output

**Format A: Phenotype Reformatting**
- Converts residuals to genome-wide BED format
- **Input:** `{celltype}_residuals.txt` (from Step 1 Option A or B)
- **Use for:** FastQTL, TensorQTL, MatrixEQTL (genome-wide caQTL mapping)

**Format B: Region Peak Filtering**
- Filters to specific genomic regions (chr7: 28-28.3 Mb, chr11: 85.05-86.2 Mb)
- **Input:** `{celltype}_filtered_raw_counts.txt` (only from Step 1 Option B)
- **Use for:** Hypothesis-driven locus analysis, region-specific comparisons

#### Alternative Pseudobulk Pipeline: Explicit Batch Correction (Multiome Dataset)
- Complete standalone pipeline with explicit batch correction using limma's `removeBatchEffect` or ComBat-seq
- **Input:** Qc'ed Seurat object`{celltype}_qced.rds` and pseudobulk peak counts `{celltype}.rds`
- **Use when:** Strong batch effects visible in PCA/t-SNE, many small fragmented batches, batch confounds with biology
- **Note:** From different dataset (multiome) but demonstrates alternative batch correction approach

---

### Key Features:
- Blacklist region filtering (ENCODE hg38)
- Technical QC covariate adjustment (TSS enrichment, nucleosome signal, sequencing depth)
- TMM normalization and expression filtering
- Log-transformation of count-based covariates
- Flexible batch handling (covariate vs explicit correction)

#### Pipeline Outputs:

**From Step 1:**
- `{celltype}_residuals.txt`: Covariate-adjusted residuals (log2-CPM scale)
- `{celltype}_results.rds`: Complete analysis results
- `{celltype}_summary.txt`: QC summary and filtering statistics
- `{celltype}_variable_explanation.txt`: Covariate documentation (Option A only)
- `{celltype}_filtered_raw_counts.txt`: TMM-normalized counts (Option B only)

**From Step 2, Format A:**
- `{celltype}_kellis_snatac_phenotype.bed.gz`: Genome-wide QTL-ready BED file

**From Step 2, Format B:**
- `{celltype}_filtered_regions_of_interest.txt`: Region-specific count data (chr7, chr11)
- `{celltype}_filtered_regions_of_interest_summary.txt`: Peak metadata and statistics

**From Alternative Pseudobulk Pipeline: Multiome with Batch Correction:**
- `{celltype}_residuals.txt`: Batch-corrected residuals (log2-CPM scale)
- `{celltype}_results.rds`: Complete results with batch_adjusted_counts

---

### Input Files
Input files needed to run this pipeline can be downloaded [here](https://drive.google.com/drive/folders/1UzJuHN8SotMn-PJTBp9uGShD25YxapKr?usp=drive_link).

#### Before you start, let's set up your working path.

In [1]:
input_dir <- "/restricted/projectnb/xqtl/jaempawi/atac_seq/kellis_data"  #set your input directory
output_dir <- "/restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis" #set your output directory

## Step 1: Pseudobulk QC with batch as covariates

This preprocessing workflow offers **two approaches** depending on whether you want to regress out biological covariates:

---
### Option A: Pseudobulk QC WITH Biological Variation(Standard QTL Analysis)

Use this option when you want residuals adjusted for all technical AND biological covariates (sex, age, PMI).

**Input:**
- Pseudobulk peak counts (in 1_files_with_sampleid folder): `pseudobulk_peaks_counts{celltype}_50nuc.csv.gz`
- Cell metadata (in 1_files_with_sampleid folder): `metadata_{celltype}_50nuc.csv`
- Sample covariates: `rosmap_cov.txt`
- hg38 blacklist: `hg38-blacklist.v2.bed.gz`

**Process:**
1. Loads pseudobulk peak count matrix and metadata per cell type
2. Calculates technical QC metrics per sample:
   - `log_n_nuclei`: Log-transformed number of nuclei
   - `med_nucleosome_signal`: Median nucleosome signal
   - `med_tss_enrich`: Median TSS enrichment score
   - `log_med_n_tot_fragment`: Log-transformed median total fragments (sequencing depth)
   - `log_total_unique_peaks`: Log-transformed count of unique peaks detected
3. Filters blacklisted genomic regions using `foverlaps()`
4. Merges with demographic covariates (msex, age_death, pmi, study)
5. Applies expression filtering with `filterByExpr()`:
   - `min.count = 2`: Minimum 2 reads in at least one sample
   - `min.total.count = 15`: Minimum 15 total reads across all samples
   - `min.prop = 0.1`: Peak must be expressed in ≥10% of samples
6. TMM normalization with `calcNormFactors()`
7. Handles sequencingBatch as a covariate (not batch-corrected)
8. Fits linear model using `voom()` and `lmFit()`:

   ```r
   model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment + log_total_unique_peaks + sequencingBatch_factor + msex + age_death + pmi + study  
   ```

9. Calculates residuals adjusted for ALL covariates (technical + biological)
10. Computes final adjusted data using predictOffset(): offset + residuals
- `offset`: Predicted expression at median/reference covariate values
- `residuals`: Unexplained variation after removing all covariate effects

**Output:** `output/2_residuals/{celltype}/`

- `{celltype}_residuals.txt`: Final covariate-adjusted peak accessibility (log2-CPM scale)
- `{celltype}_results.rds`: Complete analysis results (DGEList, fit object, design matrix)
- `{celltype}_summary.txt`: Filtering statistics and QC summary
- `{celltype}_variable_explanation.txt`: Detailed covariate documentation

**Key Variables Regressed Out**:

- Technical: sequencing depth, nuclei count, nucleosome signal, TSS enrichment, batch
- Biological: sex (msex), age at death (age_death), post-mortem interval (pmi), study cohort


#### Load libaries

In [2]:
library(data.table)
library(stringr)
library(dplyr)
library(edgeR)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: limma



In [3]:
# Set cell type and create output directory
celltype <- "Astro"  # Change this for different cell types eg. Exc, Inh, Mic, Oligo, OPC
cat("Processing celltype:", celltype, "\n")

out_dir <- paste0(file.path(output_dir,"2_residuals/", celltype))
dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)


Processing celltype: Astro 


#### Create predictOffset function 

In [4]:
predictOffset <- function(fit) {
  # Define which variables are factors and which are continuous
  usedFactors <- c("sequencingBatch", "study") 
  usedContinuous <- c("log_n_nuclei", "med_nucleosome_signal", "med_tss_enrich", "log_med_n_tot_fragment",
                      "log_total_unique_peaks", "msex", "age_death", "pmi")
  
  # Filter to only use variables actually in the design matrix
  usedFactors <- usedFactors[sapply(usedFactors, function(f) any(grepl(paste0("^", f), colnames(fit$design))))]
  usedContinuous <- usedContinuous[sapply(usedContinuous, function(f) any(grepl(paste0("^", f), colnames(fit$design))))]
  
  # Get indices for factor and continuous variables
  facInd <- unlist(lapply(as.list(usedFactors), 
                         function(f) {return(grep(paste0("^", f), 
                                                colnames(fit$design)))}))
  contInd <- unlist(lapply(as.list(usedContinuous), 
                          function(f) {return(grep(paste0("^", f), 
                                                 colnames(fit$design)))}))
  
  # Add the intercept
  all_indices <- c(1, facInd, contInd)
  
  # Verify design matrix structure (using sorted indices to avoid duplication warning)
  all_indices_sorted <- sort(unique(all_indices))
  stopifnot(all(all_indices_sorted %in% 1:ncol(fit$design)))
  
  # Create new design matrix with median values
  D <- fit$design
  D[, facInd] <- 0  # Set all factor levels to reference level
  
  # For continuous variables, set to median value
  if (length(contInd) > 0) {
    medContVals <- apply(D[, contInd, drop=FALSE], 2, median)
    for (i in 1:length(medContVals)) {
      D[, names(medContVals)[i]] <- medContVals[i]
    }
  }
  
  # Calculate offsets
  stopifnot(all(colnames(coefficients(fit)) == colnames(D)))
  offsets <- apply(coefficients(fit), 1, function(c) {
    return(D %*% c)
  })
  offsets <- t(offsets)
  colnames(offsets) <- rownames(fit$design)
  
  return(offsets)
}

#### Load input

In [5]:
meta <- fread(file.path(input_dir, "1_files_with_sampleid", paste0("metadata_", celltype, "_50nuc.csv")))
peak_data <- fread(file.path(input_dir, "1_files_with_sampleid", paste0("pseudobulk_peaks_counts", celltype, "_50nuc.csv.gz")))

cat("Loaded metadata with", nrow(meta), "samples and peak data with", nrow(peak_data), "peaks\n")

# Extract peak_id and set as rownames
peak_id <- peak_data$peak_id
peak_data <- peak_data[, -1, with = FALSE]  # Remove peak_id column
peak_matrix <- as.matrix(peak_data)
rownames(peak_matrix) <- peak_id

head(meta)
head(peak_data)

Loaded metadata with 82 samples and peak data with 531489 peaks


individualID,sampleid,sequencingBatch,main_cell_type,avg.pct.read.in.peak.ct,med.nucleosome_signal.ct,med.n_tot_fragment.ct,med.tss.enrich.ct,n.nuclei
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
R1042011,SM-CJK5G,191203Kel,Astro,0.3939189,0.7894187,10923.00,0.3762771,409
R1154454,SM-CTDQN,191203Kel,Astro,0.2557693,0.7786428,23144.00,0.2516681,144
R1213305,SM-CJEIE,191203Kel,Astro,0.3277831,0.8077042,16094.78,0.2896403,630
R1407047,SM-CTEM5,191203Kel,Astro,0.3361316,0.8275109,59451.00,0.3266785,189
R1609849,SM-CJJ27,191203Kel,Astro,0.2857020,0.7868788,7522.00,0.2688059,186
R1617674,SM-CJIWT,191203Kel,Astro,0.1934420,0.7879911,33724.00,0.1702281,141


SM-CJK5G,SM-CTDQN,SM-CJEIE,SM-CTEM5,SM-CJJ27,SM-CJIWT,SM-CTEEG,ROS11430815,SM-CJGLG,SM-CJIXU,⋯,R9395022,SM-CJIX5,SM-CJEGU,SM-CJIYH,SM-CJGMS,SM-CTEGU,SM-CTEFJ,SM-CJEJU,SM-CTEGT,SM-CJIZE
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
4,0,0,2,0,0,0,2,2,0,⋯,1,0,0,0,1,0,0,1,0,0
20,12,45,36,1,7,9,30,16,5,⋯,13,5,7,10,3,6,11,10,18,5
8,1,3,6,0,6,11,1,1,3,⋯,5,2,0,1,3,0,2,3,5,4
0,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
15,4,15,9,2,3,16,8,3,5,⋯,5,6,5,5,5,2,6,12,7,7
33,4,55,70,4,10,26,21,22,5,⋯,30,15,8,21,5,20,35,26,48,6


#### Process technical variables from meta data


In [6]:
# Column name normalization (for easier handling)
meta_clean <- meta %>%
  rename(
    med_nucleosome_signal = med.nucleosome_signal.ct,
    med_tss_enrich = med.tss.enrich.ct,
    med_n_tot_fragment = med.n_tot_fragment.ct,
    n_nuclei = n.nuclei
  )

# Calculate peak metrics - total unique peaks per sample
peak_metrics <- data.frame(
  sampleid = colnames(peak_matrix),
  total_unique_peaks = colSums(peak_matrix > 0)
) %>%
  mutate(log_total_unique_peaks = log(total_unique_peaks + 1))

####  Process peaks

In [7]:
# Process peak coordinates
peak_df <- data.table(
  peak_name = peak_id,
  chr = sapply(strsplit(peak_id, "-"), `[`, 1),
  start = as.integer(sapply(strsplit(peak_id, "-"), `[`, 2)),
  end = as.integer(sapply(strsplit(peak_id, "-"), `[`, 3)),
  stringsAsFactors = FALSE
)

# Verify peak coordinates were extracted correctly
cat("Sample of peak coordinates:\n")
print(head(peak_df))

# Load blacklist
blacklist_file <- file.path(input_dir,"hg38-blacklist.v2.bed.gz")
if (file.exists(blacklist_file)) {
  blacklist_df <- fread(blacklist_file)
  if (ncol(blacklist_df) >= 4) {
    colnames(blacklist_df)[1:4] <- c("chr", "start", "end", "label")
  } else {
    colnames(blacklist_df)[1:3] <- c("chr", "start", "end")
  }
  
  # Filter blacklisted peaks
  setkey(blacklist_df, chr, start, end)
  setkey(peak_df, chr, start, end)
  overlapping_peaks <- foverlaps(peak_df, blacklist_df, nomatch=0)
  blacklisted_peaks <- unique(overlapping_peaks$peak_name)
  cat("Number of blacklisted peaks:", length(blacklisted_peaks), "\n")
  
  filtered_peak_idx <- !(peak_id %in% blacklisted_peaks)
  filtered_peak <- peak_matrix[filtered_peak_idx, ]
  cat("Number of peaks after blacklist filtering:", nrow(filtered_peak), "\n")
} else {
  cat("Warning: Blacklist file not found at", blacklist_file, "\n")
  cat("Proceeding without blacklist filtering\n")
  filtered_peak <- peak_matrix
}

Sample of peak coordinates:
            peak_name    chr  start    end
               <char> <char>  <int>  <int>
1: chr1-181293-181565   chr1 181293 181565
2: chr1-190726-191626   chr1 190726 191626
3: chr1-629712-630662   chr1 629712 630662
4: chr1-631261-631470   chr1 631261 631470
5: chr1-633891-634506   chr1 633891 634506
6: chr1-777873-779958   chr1 777873 779958
Number of blacklisted peaks: 2354 
Number of peaks after blacklist filtering: 529135 


#### Load covariates

In [8]:
covariates_file <- file.path(input_dir,"rosmap_cov.txt")
if (file.exists(covariates_file)) {
  covariates <- fread(covariates_file)
  # Check column names and adjust if needed
  if ('#id' %in% colnames(covariates)) {
    id_col <- '#id'
  } else if ('individualID' %in% colnames(covariates)) {
    id_col <- 'individualID'
  } else {
    cat("Warning: Could not identify ID column in covariates file. Available columns:", 
        paste(colnames(covariates), collapse=", "), "\n")
    id_col <- colnames(covariates)[1]
    cat("Using", id_col, "as ID column\n")
  }
  
  # Select relevant columns
  cov_cols <- intersect(c(id_col, 'msex', 'age_death', 'pmi', 'study'), colnames(covariates))
  covariates <- covariates[, ..cov_cols]
  
  # Merge with metadata
  meta_with_ind <- meta_clean %>%
    select(sampleid, everything())
  
  all_covs <- meta_with_ind %>%
    inner_join(peak_metrics, by = "sampleid") %>%
    inner_join(covariates, by = setNames(id_col, "sampleid"))
  
  # Impute missing values
  for (col in c("pmi", "age_death")) {
    if (col %in% colnames(all_covs) && any(is.na(all_covs[[col]]))) {
      cat("Imputing missing values for", col, "\n")
      all_covs[[col]][is.na(all_covs[[col]])] <- median(all_covs[[col]], na.rm=TRUE)
    }
  }
} else {
  cat("Warning: Covariates file", covariates_file, "not found.\n")
  cat("Proceeding with only technical variables.\n")
  all_covs <- meta_clean %>%
    inner_join(peak_metrics, by = "sampleid")
}


# Perform log transformations on necessary variables
# Add a small constant to avoid log(0)
epsilon <- 1e-6

all_covs$log_n_nuclei <- log(all_covs$n_nuclei + epsilon)
all_covs$log_med_n_tot_fragment <- log(all_covs$med_n_tot_fragment + epsilon)

# Show distribution of original and log-transformed variables
cat("\nVariable statistics before and after log transformation:\n")
for (var in c("n_nuclei", "med_n_tot_fragment")) {
  orig_var <- all_covs[[var]]
  log_var <- all_covs[[paste0("log_", var)]]
  
  cat(sprintf("%s: min=%.2f, median=%.2f, max=%.2f, SD=%.2f\n", 
              var, min(orig_var), median(orig_var), max(orig_var), sd(orig_var)))
  cat(sprintf("log_%s: min=%.2f, median=%.2f, max=%.2f, SD=%.2f\n", 
              var, min(log_var), median(log_var), max(log_var), sd(log_var)))
}

cat("Number of samples after joining:", nrow(all_covs), "\n")
cat("Sample IDs:", paste(head(all_covs$sampleid), collapse=", "), "...\n")
cat("Available covariates:", paste(colnames(all_covs), collapse=", "), "\n")



Variable statistics before and after log transformation:
n_nuclei: min=56.00, median=227.00, max=1293.00, SD=193.79
log_n_nuclei: min=4.03, median=5.42, max=7.16, SD=0.64
med_n_tot_fragment: min=2890.00, median=20306.00, max=73185.00, SD=15906.37
log_med_n_tot_fragment: min=7.97, median=9.92, max=11.20, SD=0.66
Number of samples after joining: 76 
Sample IDs: SM-CJK5G, SM-CTDQN, SM-CJEIE, SM-CTEM5, SM-CJJ27, SM-CJIWT ...
Available covariates: sampleid, individualID, sequencingBatch, main_cell_type, avg.pct.read.in.peak.ct, med_nucleosome_signal, med_n_tot_fragment, med_tss_enrich, n_nuclei, total_unique_peaks, log_total_unique_peaks, msex, age_death, pmi, study, log_n_nuclei, log_med_n_tot_fragment 


#### Create DGE object

In [9]:
valid_samples <- intersect(colnames(filtered_peak), all_covs$sampleid)
cat("Number of valid samples:", length(valid_samples), "\n")

all_covs_filtered <- all_covs[all_covs$sampleid %in% valid_samples, ]
filtered_peak_filtered <- filtered_peak[, valid_samples]

dge <- DGEList(
  counts = filtered_peak_filtered,
  samples = all_covs_filtered
)
rownames(dge$samples) <- dge$samples$sampleid

Number of valid samples: 76 


#### Filter low counts and normalize

In [10]:
cat("Number of peaks before filtering:", nrow(dge), "\n")
keep <- filterByExpr(dge, 
                   min.count = 2,     # for one sample, min reads 
                   min.total.count = 15, # min reads overall
                   min.prop = 0.1) 

dge <- dge[keep, , keep.lib.sizes=FALSE]
cat("Number of peaks after filtering:", nrow(dge), "\n") #1368 in mic,2491 in Ast
dge <- calcNormFactors(dge, method="TMM")


Number of peaks before filtering: 529135 


Warning message in filterByExpr.DGEList(dge, min.count = 2, min.total.count = 15, :
“All samples appear to belong to the same group.”


Number of peaks after filtering: 323638 


####  Handle batch as technical variable

In [11]:
# We'll handle batch as a technical variable rather than doing batch adjustment
cat("Handling sequencingBatch as a technical variable\n")

# Check batch information
batches <- dge$samples$sequencingBatch
cat("Found", length(unique(batches)), "unique batches\n")

# Check batch size
batch_counts <- table(batches)
cat("Batch sizes:\n")
print(batch_counts)

# Convert sequencingBatch to factor with at least 2 levels
if (length(unique(batches)) < 2) {
  cat("Only one batch found. Adding dummy batch for model compatibility.\n")
  # Create a dummy batch factor to avoid model errors
  dge$samples$sequencingBatch_factor <- factor(rep("batch1", ncol(dge)))
} else {
  # Use the existing batch information
  dge$samples$sequencingBatch_factor <- factor(dge$samples$sequencingBatch)
}


Handling sequencingBatch as a technical variable
Found 2 unique batches
Batch sizes:
batches
190820Kel 191203Kel 
        4        72 


#### Create model and run voom

In [12]:
# Define the model based on available covariates - using log-transformed variables
if (all(c("msex", "age_death", "pmi", "study") %in% colnames(dge$samples))) {
  # Full model with all covariates
  cat("Using full model with demographic and technical covariates\n")
  model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment  +
    log_total_unique_peaks + sequencingBatch_factor + 
    msex + age_death + pmi + study
} else {
  # Technical variables only model
  cat("Using model with technical covariates only\n")
  model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment  +
    log_total_unique_peaks + sequencingBatch_factor
}

# Print the model formula
cat("Model formula:", deparse(model), "\n")

# Check for factor variables with only one level
for (col in colnames(dge$samples)) {
  if (is.factor(dge$samples[[col]]) && nlevels(dge$samples[[col]]) < 2) {
    cat("Warning: Factor variable", col, "has only one level. Converting to character.\n")
    dge$samples[[col]] <- as.character(dge$samples[[col]])
  }
}

# Create design matrix with error checking
tryCatch({
  design <- model.matrix(model, data=dge$samples)
  cat("Successfully created design matrix with", ncol(design), "columns\n")
}, error = function(e) {
  cat("Error in creating design matrix:", e$message, "\n")
  cat("Attempting to fix model formula...\n")
  
  # Check each term in the model
  all_terms <- all.vars(model)
  valid_terms <- character(0)
  
  for (term in all_terms) {
    if (term %in% colnames(dge$samples)) {
      # Check if it's a factor with at least 2 levels
      if (is.factor(dge$samples[[term]])) {
        if (nlevels(dge$samples[[term]]) >= 2) {
          valid_terms <- c(valid_terms, term)
        } else {
          cat("Skipping factor", term, "with only", nlevels(dge$samples[[term]]), "level\n")
        }
      } else {
        # Non-factor variables are fine
        valid_terms <- c(valid_terms, term)
      }
    } else {
      cat("Variable", term, "not found in sample data\n")
    }
  }
  
  # Create a simplified model with valid terms
  if (length(valid_terms) > 0) {
    model_str <- paste("~", paste(valid_terms, collapse = " + "))
    model <- as.formula(model_str)
    cat("New model formula:", model_str, "\n")
    design <- model.matrix(model, data=dge$samples)
    cat("Successfully created design matrix with", ncol(design), "columns\n")
  } else {
    stop("Could not create a valid model with the available variables")
  }
})

# Check if the design matrix is full rank
if (!is.fullrank(design)) {
  cat("Design matrix is not full rank. Adjusting...\n")
  # Find and remove the problematic columns
  qr_res <- qr(design)
  design <- design[, qr_res$pivot[1:qr_res$rank]]
  cat("Adjusted design matrix columns:", ncol(design), "\n")
}

# Run voom and fit model
v <- voom(dge, design, plot=FALSE) #logCPM
fit <- lmFit(v, design)
fit <- eBayes(fit)

# Calculate offset and residuals
cat("Calculating offsets and residuals...\n")
offset <- predictOffset(fit)
resids <- residuals(fit, y=v)

# Verify dimensions
stopifnot(all(rownames(offset) == rownames(resids)) &
          all(colnames(offset) == colnames(resids)))

# Final adjusted data
stopifnot(all(dim(offset) == dim(resids)))
stopifnot(all(colnames(offset) == colnames(resids)))

final_data <- offset + resids

Using full model with demographic and technical covariates
Model formula: ~log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment +      log_total_unique_peaks + sequencingBatch_factor + msex +      age_death + pmi + study 
Successfully created design matrix with 11 columns
Calculating offsets and residuals...


#### Save results

In [13]:
# Save results
saveRDS(list(
  dge = dge,
  offset = offset,
  residuals = resids,
  final_data = final_data,
  valid_samples = colnames(dge),
  design = design,
  fit = fit,
  model = model
), file = file.path(out_dir, paste0(celltype,"_results.rds")))

# Write final residual data to file
write.table(final_data,
            file = file.path(out_dir, paste0(celltype,"_residuals.txt")), 
            quote=FALSE, sep="\t", row.names=TRUE, col.names=TRUE)

# Write summary statistics
sink(file = file.path(out_dir, paste0(celltype, "_summary.txt")))
cat("*** Processing Summary for", celltype, "***\n\n")
cat("Original peak count:", length(peak_id), "\n")
cat("Peaks after blacklist filtering:", nrow(filtered_peak), "\n")
cat("Peaks after expression filtering:", nrow(dge), "\n\n")
cat("Number of samples:", ncol(dge), "\n")
cat("\nTechnical Variables Used:\n")
cat("- log_n_nuclei: Log-transformed number of nuclei per sample\n")
cat("- med_nucleosome_signal: Median nucleosome signal\n")
cat("- med_tss_enrich: Median TSS enrichment\n")
cat("- log_med_n_tot_fragment: Log-transformed median number of total fragments\n")
cat("- log_total_unique_peaks: Log-transformed count of unique peaks per sample\n")
cat("\nDemographic Variables Used:\n")
cat("- msex: Sex (male=1, female=0)\n")
cat("- age_death: Age at death\n")
cat("- pmi: Post-mortem interval\n")
cat("- study: Study cohort\n")
sink()

# Write an additional explanation file about the variables and log transformation
sink(file = file.path(out_dir, paste0(celltype,"_variable_explanation.txt")))
cat("# ATAC-seq Technical Variables Explanation\n\n")

cat("## Why Log Transformation?\n")
cat("Log transformation is applied to certain variables for several reasons:\n")
cat("1. To make the distribution more symmetric and closer to normal\n")
cat("2. To stabilize variance across the range of values\n")
cat("3. To match the scale of voom-transformed peak counts, which are on log2-CPM scale\n")
cat("4. To be consistent with the approach used in related studies like haQTL\n\n")

cat("## Variables and Their Meanings\n\n")

cat("### Technical Variables\n")
cat("- n_nuclei: Number of nuclei that contributed to this pseudobulk sample\n")
cat("  * Log-transformed because count data typically has a right-skewed distribution\n\n")

cat("- med_n_tot_fragment: Median number of total fragments per cell\n")
cat("  * Represents sequencing depth\n")
cat("  * Log-transformed because sequencing depth typically has exponential effects\n\n")

cat("- total_unique_peaks: Number of unique peaks detected in each sample\n")
cat("  * Log-transformed similar to 'TotalNumPeaks' in haQTL pipeline\n\n")

cat("- med_nucleosome_signal: Median nucleosome signal\n")
cat("  * Measures the degree of nucleosome positioning\n")
cat("  * Not log-transformed as it's already a ratio/normalized metric\n\n")

cat("- med_tss_enrich: Median transcription start site enrichment score\n")
cat("  * Indicates the quality of the ATAC-seq data\n")
cat("  * Not log-transformed as it's already a ratio/normalized metric\n\n")

cat("### Demographic Variables\n")
cat("- msex: Sex (male=1, female=0)\n")
cat("- age_death: Age at death\n")
cat("- pmi: Post-mortem interval (time between death and tissue collection)\n")
cat("- study: Study cohort (ROSMAP, MAP, ROS)\n\n")

cat("## Relationship to voom Transformation\n")
cat("The voom transformation converts count data to log2-CPM (counts per million) values ")
cat("and estimates the mean-variance relationship. By log-transforming certain technical ")
cat("covariates, we ensure they're on a similar scale to the transformed expression data, ")
cat("which can improve the fit of the linear model used for removing unwanted variation.\n")
sink()

cat("Processing completed. Results and documentation saved to:", out_dir, "\n")

Processing completed. Results and documentation saved to: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals//Astro 


### Option B: Pseudobulk QC WITHOUT Biological Variation (noBIOvar)
Use this option when you want to preserve biological variation (e.g., for comparing across ages/sexes or region-specific analyses).

**Input:** (Same as Option A)
- Pseudobulk peak counts (in `1_files_with_sampleid` folder): `pseudobulk_peaks_counts{celltype}_50nuc.csv.gz`
- Cell metadata (in `1_files_with_sampleid` folder): `metadata_{celltype}_50nuc.csv`
- Sample covariates: `rosmap_cov.txt`
- hg38 blacklist: `hg38-blacklist.v2.bed.gz`

**Process:**
1. Loads pseudobulk peak count matrix and metadata per cell type
2. Calculates technical QC metrics per sample:
   - `log_n_nuclei`: Log-transformed number of nuclei
   - `med_nucleosome_signal`: Median nucleosome signal
   - `med_tss_enrich`: Median TSS enrichment score
   - `log_med_n_tot_fragment`: Log-transformed median total fragments (sequencing depth)
   - `log_total_unique_peaks`: Log-transformed count of unique peaks detected
3. Filters blacklisted genomic regions using `foverlaps()`
4. Merges with demographic covariates (msex, age_death, pmi, study)
5. Applies expression filtering with `filterByExpr()`:
   - `min.count = 2`: Minimum 2 reads in at least one sample
   - `min.total.count = 15`: Minimum 15 total reads across all samples
   - `min.prop = 0.1`: Peak must be expressed in ≥10% of samples
6. TMM normalization with `calcNormFactors()`
7. Saves **filtered raw counts** without covariate adjustment

**Key Difference:** 
- Does NOT regress out msex or age_death
- No residual calculation performed (voom/lmFit section commented out)
- Only saves TMM-normalized, filtered count matrix

**Model formula (if residuals were computed):**
```r
model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment + log_total_unique_peaks + med_peakwidth + sequencingBatch_factor + pmi + study

```
Note: The voom/residual calculation section is commented out; only filtered counts are saved

**Output:** `output/2_residuals/{celltype}/`

`{celltype}_filtered_raw_counts.txt`: TMM-normalized, filtered peak counts without biological covariate adjustment

**Key Variables NOT Regressed:**
- Sex (msex)
- Age at death (age_death)

#### Load libraries

In [14]:
library(data.table)
library(stringr)
library(dplyr)
library(edgeR)

#### Load input

In [15]:
# Set cell type and create output directory
#args <- commandArgs(trailingOnly = TRUE)
#celltype <- args[1]  # First argument is the cell type
celltype <- "Exc"  # Change this for different cell types
cat("Processing celltype:", celltype, "\n")

out_dir <- paste0(file.path(output_dir,"2_residuals/", celltype))
dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

Processing celltype: Exc 


#### Create predictOffset function 

In [16]:
predictOffset <- function(fit) {
  # Define which variables are factors and which are continuous
  usedFactors <- c("sequencingBatch", "study") 
  usedContinuous <- c("log_n_nuclei", "med_nucleosome_signal", "med_tss_enrich", "log_med_n_tot_fragment",
                      "log_total_unique_peaks", "med_peakwidth", "pmi")
  
  # Filter to only use variables actually in the design matrix
  usedFactors <- usedFactors[sapply(usedFactors, function(f) any(grepl(paste0("^", f), colnames(fit$design))))]
  usedContinuous <- usedContinuous[sapply(usedContinuous, function(f) any(grepl(paste0("^", f), colnames(fit$design))))]
  
  # Get indices for factor and continuous variables
  facInd <- unlist(lapply(as.list(usedFactors), 
                         function(f) {return(grep(paste0("^", f), 
                                                colnames(fit$design)))}))
  contInd <- unlist(lapply(as.list(usedContinuous), 
                          function(f) {return(grep(paste0("^", f), 
                                                 colnames(fit$design)))}))
  
  # Add the intercept
  all_indices <- c(1, facInd, contInd)
  
  # Verify design matrix structure (using sorted indices to avoid duplication warning)
  all_indices_sorted <- sort(unique(all_indices))
  stopifnot(all(all_indices_sorted %in% 1:ncol(fit$design)))
  
  # Create new design matrix with median values
  D <- fit$design
  D[, facInd] <- 0  # Set all factor levels to reference level
  
  # For continuous variables, set to median value
  if (length(contInd) > 0) {
    medContVals <- apply(D[, contInd, drop=FALSE], 2, median)
    for (i in 1:length(medContVals)) {
      D[, names(medContVals)[i]] <- medContVals[i]
    }
  }
  
  # Calculate offsets
  stopifnot(all(colnames(coefficients(fit)) == colnames(D)))
  offsets <- apply(coefficients(fit), 1, function(c) {
    return(D %*% c)
  })
  offsets <- t(offsets)
  colnames(offsets) <- rownames(fit$design)
  
  return(offsets)
}

#### Load input

In [17]:
meta <- fread(paste0(file.path(input_dir, "1_files_with_sampleid/metadata_"), celltype, "_50nuc.csv"))
peak_data <- fread(file.path(input_dir,"1_files_with_sampleid", paste0("pseudobulk_peaks_counts", celltype, "_50nuc.csv.gz")))

cat("Loaded metadata with", nrow(meta), "samples and peak data with", nrow(peak_data), "peaks\n")

# Extract peak_id and set as rownames
peak_id <- peak_data$peak_id
peak_data <- peak_data[, -1, with = FALSE]  # Remove peak_id column
peak_matrix <- as.matrix(peak_data)
rownames(peak_matrix) <- peak_id

Loaded metadata with 90 samples and peak data with 531489 peaks


#### Process technical variables from meta data

In [18]:
# Column name normalization (for easier handling)
meta_clean <- meta %>%
  rename(
    med_nucleosome_signal = med.nucleosome_signal.ct,
    med_tss_enrich = med.tss.enrich.ct,
    med_n_tot_fragment = med.n_tot_fragment.ct,
    n_nuclei = n.nuclei
  )

# Calculate peak metrics - total unique peaks per sample and median peak width
peak_metrics <- data.frame(
  sampleid = colnames(peak_matrix),
  total_unique_peaks = colSums(peak_matrix > 0)
) %>%
  mutate(log_total_unique_peaks = log(total_unique_peaks + 1))

# Calculate median peak width for each sample using count as weight
calculate_median_peakwidth <- function(peak_matrix, peak_info) {
  # Create a data frame with peak widths
  peak_widths <- peak_info$end - peak_info$start
  
  # Initialize a vector to store median peak widths
  median_peak_widths <- numeric(ncol(peak_matrix))
  names(median_peak_widths) <- colnames(peak_matrix)
  
  # For each sample, calculate the weighted median peak width
  for (i in 1:ncol(peak_matrix)) {
    sample_counts <- peak_matrix[, i]
    # Only consider peaks with counts > 0
    idx <- which(sample_counts > 0)
    
    if (length(idx) > 0) {
      # Method 1: Use counts as weights
      weights <- sample_counts[idx]
      # Repeat each peak width by its count for weighted calculation
      all_widths <- rep(peak_widths[idx], times=weights)
      median_peak_widths[i] <- median(all_widths)
    } else {
      median_peak_widths[i] <- NA
    }
  }
  
  return(median_peak_widths)
}

# Calculate median peak width for each sample
# Note: Using the peak_df that was created earlier for blacklist filtering
median_peakwidths <- calculate_median_peakwidth(peak_matrix, data.frame(
  start = as.integer(sapply(strsplit(peak_id, "-"), `[`, 2)),
  end = as.integer(sapply(strsplit(peak_id, "-"), `[`, 3))
))

# Add median peak width to peak metrics
peak_metrics$med_peakwidth <- median_peakwidths

#### Process peaks

In [19]:
# Process peak coordinates
peak_df <- data.table(
  peak_name = peak_id,
  chr = sapply(strsplit(peak_id, "-"), `[`, 1),
  start = as.integer(sapply(strsplit(peak_id, "-"), `[`, 2)),
  end = as.integer(sapply(strsplit(peak_id, "-"), `[`, 3)),
  stringsAsFactors = FALSE
)

# Verify peak coordinates were extracted correctly
cat("Sample of peak coordinates:\n")
print(head(peak_df))

# Load blacklist
blacklist_file <- file.path(input_dir,"hg38-blacklist.v2.bed.gz")
if (file.exists(blacklist_file)) {
  blacklist_df <- fread(blacklist_file)
  if (ncol(blacklist_df) >= 4) {
    colnames(blacklist_df)[1:4] <- c("chr", "start", "end", "label")
  } else {
    colnames(blacklist_df)[1:3] <- c("chr", "start", "end")
  }
  
  # Filter blacklisted peaks
  setkey(blacklist_df, chr, start, end)
  setkey(peak_df, chr, start, end)
  overlapping_peaks <- foverlaps(peak_df, blacklist_df, nomatch=0)
  blacklisted_peaks <- unique(overlapping_peaks$peak_name)
  cat("Number of blacklisted peaks:", length(blacklisted_peaks), "\n")
  
  filtered_peak_idx <- !(peak_id %in% blacklisted_peaks)
  filtered_peak <- peak_matrix[filtered_peak_idx, ]
  cat("Number of peaks after blacklist filtering:", nrow(filtered_peak), "\n")
} else {
  cat("Warning: Blacklist file not found at", blacklist_file, "\n")
  cat("Proceeding without blacklist filtering\n")
  filtered_peak <- peak_matrix
}


Sample of peak coordinates:
            peak_name    chr  start    end
               <char> <char>  <int>  <int>
1: chr1-181293-181565   chr1 181293 181565
2: chr1-190726-191626   chr1 190726 191626
3: chr1-629712-630662   chr1 629712 630662
4: chr1-631261-631470   chr1 631261 631470
5: chr1-633891-634506   chr1 633891 634506
6: chr1-777873-779958   chr1 777873 779958
Number of blacklisted peaks: 2354 
Number of peaks after blacklist filtering: 529135 


#### Load covariates

In [20]:
covariates_file <- file.path(input_dir,'rosmap_cov.txt')
if (file.exists(covariates_file)) {
  covariates <- fread(covariates_file)
  # Check column names and adjust if needed
  if ('#id' %in% colnames(covariates)) {
    id_col <- '#id'
  } else if ('individualID' %in% colnames(covariates)) {
    id_col <- 'individualID'
  } else {
    cat("Warning: Could not identify ID column in covariates file. Available columns:", 
        paste(colnames(covariates), collapse=", "), "\n")
    id_col <- colnames(covariates)[1]
    cat("Using", id_col, "as ID column\n")
  }
  
  # Select relevant columns - excluding msex and age_death
  cov_cols <- intersect(c(id_col, 'pmi', 'study'), colnames(covariates))
  covariates <- covariates[, ..cov_cols]
  
  # Merge with metadata
  meta_with_ind <- meta_clean %>%
    select(sampleid, everything())
  
  all_covs <- meta_with_ind %>%
    inner_join(peak_metrics, by = "sampleid") %>%
    inner_join(covariates, by = setNames(id_col, "sampleid"))
  
  # Impute missing values
  for (col in c("pmi")) {
    if (col %in% colnames(all_covs) && any(is.na(all_covs[[col]]))) {
      cat("Imputing missing values for", col, "\n")
      all_covs[[col]][is.na(all_covs[[col]])] <- median(all_covs[[col]], na.rm=TRUE)
    }
  }
} else {
  cat("Warning: Covariates file", covariates_file, "not found.\n")
  cat("Proceeding with only technical variables.\n")
  all_covs <- meta_clean %>%
    inner_join(peak_metrics, by = "sampleid")
}


# Perform log transformations on necessary variables
# Add a small constant to avoid log(0)
epsilon <- 1e-6

all_covs$log_n_nuclei <- log(all_covs$n_nuclei + epsilon)
all_covs$log_med_n_tot_fragment <- log(all_covs$med_n_tot_fragment + epsilon)

# Show distribution of original and log-transformed variables
cat("\nVariable statistics before and after log transformation:\n")
for (var in c("n_nuclei", "med_n_tot_fragment")) {
  orig_var <- all_covs[[var]]
  log_var <- all_covs[[paste0("log_", var)]]
  
  cat(sprintf("%s: min=%.2f, median=%.2f, max=%.2f, SD=%.2f\n", 
              var, min(orig_var), median(orig_var), max(orig_var), sd(orig_var)))
  cat(sprintf("log_%s: min=%.2f, median=%.2f, max=%.2f, SD=%.2f\n", 
              var, min(log_var), median(log_var), max(log_var), sd(log_var)))
}

cat("Number of samples after joining:", nrow(all_covs), "\n")
cat("Sample IDs:", paste(head(all_covs$sampleid), collapse=", "), "...\n")
cat("Available covariates:", paste(colnames(all_covs), collapse=", "), "\n")


Variable statistics before and after log transformation:
n_nuclei: min=77.00, median=1762.00, max=7024.00, SD=1275.90
log_n_nuclei: min=4.34, median=7.47, max=8.86, SD=0.88
med_n_tot_fragment: min=3234.00, median=21072.00, max=133932.50, SD=20162.62
log_med_n_tot_fragment: min=8.08, median=9.96, max=11.81, SD=0.73
Number of samples after joining: 83 
Sample IDs: SM-CJK5G, SM-CTDQN, SM-CJEIE, SM-CTEM5, SM-CJJ27, SM-CJIWT ...
Available covariates: sampleid, individualID, sequencingBatch, main_cell_type, avg.pct.read.in.peak.ct, med_nucleosome_signal, med_n_tot_fragment, med_tss_enrich, n_nuclei, total_unique_peaks, log_total_unique_peaks, med_peakwidth, pmi, study, log_n_nuclei, log_med_n_tot_fragment 


#### Create DGE object

In [21]:
valid_samples <- intersect(colnames(filtered_peak), all_covs$sampleid)
cat("Number of valid samples:", length(valid_samples), "\n")

all_covs_filtered <- all_covs[all_covs$sampleid %in% valid_samples, ]
filtered_peak_filtered <- filtered_peak[, valid_samples]

dge <- DGEList(
  counts = filtered_peak_filtered,
  samples = all_covs_filtered
)
rownames(dge$samples) <- dge$samples$sampleid

Number of valid samples: 83 


#### Filter low counts and normalize

In [22]:
cat("Number of peaks before filtering:", nrow(dge), "\n")
keep <- filterByExpr(dge, 
                   min.count = 5,     # for one sample, min reads 
                   min.total.count = 15, # min reads overall
                   min.prop = 0.1) 

dge <- dge[keep, , keep.lib.sizes=FALSE]
cat("Number of peaks after filtering:", nrow(dge), "\n") #1368 in mic,2491 in Ast

# Save filtered raw count data
filtered_raw_counts <- dge$counts
write.table(filtered_raw_counts,
            file = file.path(out_dir, paste0(celltype, "_filtered_raw_counts.txt")), 
            quote=FALSE, sep="\t", row.names=TRUE, col.names=TRUE)
cat("Saved filtered raw counts to", file.path(out_dir, paste0(celltype, "_filtered_raw_counts.txt")), "\n")

Number of peaks before filtering: 529135 


Warning message in filterByExpr.DGEList(dge, min.count = 5, min.total.count = 15, :
“All samples appear to belong to the same group.”


Number of peaks after filtering: 521515 
Saved filtered raw counts to /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals//Exc/Exc_filtered_raw_counts.txt 


#### Handle batch as technical variable

In [23]:
dge <- calcNormFactors(dge, method="TMM")
# We'll handle batch as a technical variable rather than doing batch adjustment
cat("Handling sequencingBatch as a technical variable\n")

# Check batch information
batches <- dge$samples$sequencingBatch
cat("Found", length(unique(batches)), "unique batches\n")

# Check batch size
batch_counts <- table(batches)
cat("Batch sizes:\n")
print(batch_counts)

# Convert sequencingBatch to factor with at least 2 levels
if (length(unique(batches)) < 2) {
  cat("Only one batch found. Adding dummy batch for model compatibility.\n")
  # Create a dummy batch factor to avoid model errors
  dge$samples$sequencingBatch_factor <- factor(rep("batch1", ncol(dge)))
} else {
  # Use the existing batch information
  dge$samples$sequencingBatch_factor <- factor(dge$samples$sequencingBatch)
}

Handling sequencingBatch as a technical variable
Found 2 unique batches
Batch sizes:
batches
190820Kel 191203Kel 
        6        77 


#### Create model and run voom

In [24]:
# Define the model based on available covariates - using log-transformed variables
# Removed msex and age_death from the model
if ("study" %in% colnames(dge$samples) && "pmi" %in% colnames(dge$samples)) {
  # Technical model with pmi and study
  cat("Using model with technical covariates plus pmi and study\n")
  model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment  +
    log_total_unique_peaks + med_peakwidth + sequencingBatch_factor + pmi + study
} else if ("pmi" %in% colnames(dge$samples)) {
  # Technical model with pmi only
  cat("Using model with technical covariates and pmi\n")
  model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment  +
    log_total_unique_peaks + med_peakwidth + sequencingBatch_factor + pmi
} else {
  # Technical variables only model
  cat("Using model with technical covariates only\n")
  model <- ~ log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment  +
    log_total_unique_peaks + med_peakwidth + sequencingBatch_factor
}

# Print the model formula
cat("Model formula:", deparse(model), "\n")

# Check for factor variables with only one level
for (col in colnames(dge$samples)) {
  if (is.factor(dge$samples[[col]]) && nlevels(dge$samples[[col]]) < 2) {
    cat("Warning: Factor variable", col, "has only one level. Converting to character.\n")
    dge$samples[[col]] <- as.character(dge$samples[[col]])
  }
}

# Create design matrix with error checking
tryCatch({
  design <- model.matrix(model, data=dge$samples)
  cat("Successfully created design matrix with", ncol(design), "columns\n")
}, error = function(e) {
  cat("Error in creating design matrix:", e$message, "\n")
  cat("Attempting to fix model formula...\n")
  
  # Check each term in the model
  all_terms <- all.vars(model)
  valid_terms <- character(0)
  
  for (term in all_terms) {
    if (term %in% colnames(dge$samples)) {
      # Check if it's a factor with at least 2 levels
      if (is.factor(dge$samples[[term]])) {
        if (nlevels(dge$samples[[term]]) >= 2) {
          valid_terms <- c(valid_terms, term)
        } else {
          cat("Skipping factor", term, "with only", nlevels(dge$samples[[term]]), "level\n")
        }
      } else {
        # Non-factor variables are fine
        valid_terms <- c(valid_terms, term)
      }
    } else {
      cat("Variable", term, "not found in sample data\n")
    }
  }
  
  # Create a simplified model with valid terms
  if (length(valid_terms) > 0) {
    model_str <- paste("~", paste(valid_terms, collapse = " + "))
    model <- as.formula(model_str)
    cat("New model formula:", model_str, "\n")
    design <- model.matrix(model, data=dge$samples)
    cat("Successfully created design matrix with", ncol(design), "columns\n")
  } else {
    stop("Could not create a valid model with the available variables")
  }
})

# Check if the design matrix is full rank
if (!is.fullrank(design)) {
  cat("Design matrix is not full rank. Adjusting...\n")
  # Find and remove the problematic columns
  qr_res <- qr(design)
  design <- design[, qr_res$pivot[1:qr_res$rank]]
  cat("Adjusted design matrix columns:", ncol(design), "\n")
}

# Run voom and fit model
v <- voom(dge, design, plot=FALSE) #logCPM
fit <- lmFit(v, design)
fit <- eBayes(fit)

# Calculate offset and residuals
cat("Calculating offsets and residuals...\n")
offset <- predictOffset(fit)
resids <- residuals(fit, y=v)

# Verify dimensions
stopifnot(all(rownames(offset) == rownames(resids)) & all(colnames(offset) == colnames(resids)))

# Final adjusted data
stopifnot(all(dim(offset) == dim(resids)))
stopifnot(all(colnames(offset) == colnames(resids)))

final_data <- offset + resids

Using model with technical covariates plus pmi and study
Model formula: ~log_n_nuclei + med_nucleosome_signal + med_tss_enrich + log_med_n_tot_fragment +      log_total_unique_peaks + med_peakwidth + sequencingBatch_factor +      pmi + study 
Successfully created design matrix with 10 columns
Calculating offsets and residuals...


#### Save results

In [ ]:
saveRDS(list(
  dge = dge,
  offset = offset,
  residuals = resids,
  final_data = final_data,
  valid_samples = colnames(dge),
  design = design,
  fit = fit,
  model = model
), file = file.path(out_dir, paste0(celltype, "_results.rds")))

# Write final residual data to file
write.table(final_data,
            file = file.path(out_dir, paste0(celltype, "_residuals.txt")), 
            quote=FALSE, sep="\t", row.names=TRUE, col.names=TRUE)

# Write summary statistics
sink(file = file.path(out_dir, paste0(celltype, "_summary.txt")))
cat("*** Processing Summary for", celltype, "***\n\n")
cat("Original peak count:", length(peak_id), "\n")
cat("Peaks after blacklist filtering:", nrow(filtered_peak), "\n")
cat("Peaks after expression filtering:", nrow(dge), "\n\n")
cat("Number of samples:", ncol(dge), "\n")
cat("\nTechnical Variables Used:\n")
cat("- log_n_nuclei: Log-transformed number of nuclei per sample\n")
cat("- med_nucleosome_signal: Median nucleosome signal\n")
cat("- med_tss_enrich: Median TSS enrichment\n")
cat("- log_med_n_tot_fragment: Log-transformed median number of total fragments\n")
cat("- log_total_unique_peaks: Log-transformed count of unique peaks per sample\n")
cat("\nOther Variables Used:\n")
cat("- pmi: Post-mortem interval\n")
cat("- study: Study cohort\n")
sink()

# Write an additional explanation file about the variables and log transformation
sink(file = file.path(out_dir, paste0(celltype, "_variable_explanation.txt")))
cat("# ATAC-seq Technical Variables Explanation\n\n")

cat("## Why Log Transformation?\n")
cat("Log transformation is applied to certain variables for several reasons:\n")
cat("1. To make the distribution more symmetric and closer to normal\n")
cat("2. To stabilize variance across the range of values\n")
cat("3. To match the scale of voom-transformed peak counts, which are on log2-CPM scale\n")
cat("4. To be consistent with the approach used in related studies like haQTL\n\n")

cat("## Variables and Their Meanings\n\n")

cat("### Technical Variables\n")
cat("- n_nuclei: Number of nuclei that contributed to this pseudobulk sample\n")
cat("  * Log-transformed because count data typically has a right-skewed distribution\n\n")

cat("- med_n_tot_fragment: Median number of total fragments per cell\n")
cat("  * Represents sequencing depth\n")
cat("  * Log-transformed because sequencing depth typically has exponential effects\n\n")

cat("- total_unique_peaks: Number of unique peaks detected in each sample\n")
cat("  * Log-transformed similar to 'TotalNumPeaks' in haQTL pipeline\n\n")

cat("- med_peakwidth: Median width of peaks in each sample (weighted by counts)\n")
cat("  * Represents the typical size of accessible regions\n\n")

cat("- med_nucleosome_signal: Median nucleosome signal\n")
cat("  * Measures the degree of nucleosome positioning\n")
cat("  * Not log-transformed as it's already a ratio/normalized metric\n\n")

cat("- med_tss_enrich: Median transcription start site enrichment score\n")
cat("  * Indicates the quality of the ATAC-seq data\n")
cat("  * Not log-transformed as it's already a ratio/normalized metric\n\n")

cat("### Other Variables\n")
cat("- pmi: Post-mortem interval (time between death and tissue collection)\n")
cat("- study: Study cohort (ROSMAP, MAP, ROS)\n\n")

cat("## Relationship to voom Transformation\n")
cat("The voom transformation converts count data to log2-CPM (counts per million) values ")
cat("and estimates the mean-variance relationship. By log-transforming certain technical ")
cat("covariates, we ensure they're on a similar scale to the transformed expression data, ")
cat("which can improve the fit of the linear model used for removing unwanted variation.\n")
sink()

cat("Processing completed. Results and documentation saved to:", out_dir, "\n")

## Step 2: Format Output
### Format A: Phenotype Reformatting 

**Input:**
- `{celltype}_residuals.txt` from Step 1 Option A (in `2_residuals/{celltype}/`)

**Process:**
1. Reads residuals file with proper handling of peak IDs and sample columns
2. Parses peak coordinates from peak IDs (format: `chr-start-end`)
3. Converts peaks to **midpoint coordinates**:
   ```r
   midpoint = (start + end) / 2
   start = midpoint
   end = midpoint + 1
4. Creates BED format: `#chr`, `start`, `end`, `ID` (peak_id), followed by sample expression values
5. Sorts by chromosome and genomic position using `setorder(bed_data, '#chr', start, end)`
6. Writes BED file with headers
7. Compresses with `bgzip -f`

**Output:** `output/3_phenotype_processing/{celltype}`

- `{celltype}_kellis_snatac_phenotype.bed.gz`: QTL-ready BED file with peak midpoint coordinates and bgzip-compressed format

**Use Case:**
Standard caQTL (chromatin accessibility QTL) mapping where you want to identify genetic variants affecting chromatin accessibility independent of demographic factors. Ready for FastQTL, TensorQTL, or QTLtools.



#### Load libraries

In [26]:
library(data.table)
library(stringr)

#### Load input

In [27]:
# Get command line arguments
#args <- commandArgs(trailingOnly = TRUE)
#if (length(args) < 1) {
#  celltype <- "Astro"  # Default cell type
#  cat("No cell type specified, using default:", celltype, "\n")
#} else {
#  celltype <- args[1]
#  cat("Processing cell type:", celltype, "\n")
#}

celltype <- "Astro"

# Define input and output paths
reformat_input_dir <- file.path(output_dir,"2_residuals")
#output_dir <- "/home/al4225/project/kellis_snatac/output/3_phenotype_processing"
reformat_output_dir <- paste0(output_dir,"/3_phenotype_processing/", celltype)

# Create output directory if it doesn't exist
dir.create(reformat_output_dir, recursive = TRUE, showWarnings = FALSE)

# Check if input directory exists
celltype_dir <- file.path(reformat_input_dir, celltype)
if (!dir.exists(reformat_input_dir)) {
  cat("Cell type directory not found:", celltype_dir, "\n")
  cat("Using backup directory...\n")
  celltype_dir <- file.path(reformat_input_dir, "backup", celltype)
  if (!dir.exists(celltype_dir)) {
    stop("Backup directory not found either: ", celltype_dir)
  }
}

input_file <- file.path(celltype_dir, paste0(celltype, "_residuals.txt"))
output_bed <- file.path(reformat_output_dir, paste0(celltype, "_kellis_snatac_phenotype.bed"))

# Check if input file exists
if (!file.exists(input_file)) {
  stop("Input file not found: ", input_file)
}

#### Processing data

In [28]:
# Read the first line manually to get the column names
first_line <- readLines(input_file, n = 1)
col_names <- unlist(strsplit(first_line, split = "\t"))
cat("Column names from first line:", paste(head(col_names), collapse = ", "), "...\n")

# Read the residuals file using fread but skip the header
cat("Reading residuals file:", input_file, "\n")
residuals <- fread(input_file, header = FALSE, skip = 1)

# If we have an extra column compared to the header line (often happens with rownames)
if (ncol(residuals) > length(col_names)) {
  cat("File has more data columns than header columns. Assuming first column is peak IDs.\n")
  peak_ids <- residuals[[1]]
  residuals <- residuals[, -1, with = FALSE]
  # Set proper column names excluding the first one which was for peak IDs
  if (length(col_names) >= 2) {
    setnames(residuals, col_names)
  }
} else {
  # Normal case - columns match
  setnames(residuals, col_names)
  peak_ids <- residuals[[1]]
  residuals <- residuals[, -1, with = FALSE]
}

# Check that peak IDs and column names were properly extracted
cat("First few peak IDs:", paste(head(peak_ids), collapse = ", "), "\n")
cat("First few column names:", paste(head(colnames(residuals)), collapse = ", "), "\n")

# Parse peak IDs to get chromosome, start, and end
# cat("Parsing peak IDs into BED format\n")
# parsed_peaks <- data.table(
#   '#chr' = sapply(strsplit(peak_ids, "-"), `[`, 1),
#   start = as.integer(sapply(strsplit(peak_ids, "-"), `[`, 2)),
#   end = as.integer(sapply(strsplit(peak_ids, "-"), `[`, 3)),
#   ID = peak_ids  # Use peak_id as the ID column (4th column in BED)
# )


Column names from first line: SM-CJK5G, SM-CTDQN, SM-CJEIE, SM-CTEM5, SM-CJJ27, SM-CJIWT ...
Reading residuals file: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/Astro/Astro_residuals.txt 
File has more data columns than header columns. Assuming first column is peak IDs.
First few peak IDs: chr1-816945-817430, chr1-817852-818227, chr1-818626-819158, chr1-826625-827679, chr1-869475-870473, chr1-903568-904912 
First few column names: SM-CJK5G, SM-CTDQN, SM-CJEIE, SM-CTEM5, SM-CJJ27, SM-CJIWT 


#### Parse peak ID

In [29]:
# Parse peak IDs to get chromosome, start, and end
cat("Parsing peak IDs into BED format with midpoint coordinates\n")

parsed_peaks <- data.table(
  '#chr' = sapply(strsplit(peak_ids, "-"), `[`, 1),
  start = as.integer((as.integer(sapply(strsplit(peak_ids, "-"), `[`, 2)) + 
                     as.integer(sapply(strsplit(peak_ids, "-"), `[`, 3))) / 2),
  end = as.integer(((as.integer(sapply(strsplit(peak_ids, "-"), `[`, 2)) + 
                    as.integer(sapply(strsplit(peak_ids, "-"), `[`, 3))) / 2) + 1),                   
  ID = peak_ids  # Use peak_id as the ID column (4th column in BED)
)


# Add validation to ensure end > start
if (any(parsed_peaks$end <= parsed_peaks$start)) {
  cat("Warning: Found records where end <= start. Fixing...\n")
  parsed_peaks[end <= start, end := start + 1]
}



Parsing peak IDs into BED format with midpoint coordinates


#### Create BED

In [30]:
# Create BED format with all data columns
# BED format: chr, start, end, ID, followed by phenotype values with sample IDs as column names
bed_data <- cbind(parsed_peaks, residuals)

# Sort by chromosome and position
setorder(bed_data, '#chr', start, end)

# Write BED file with headers
cat("Writing BED file to:", output_bed, "\n")
fwrite(bed_data, output_bed, sep = "\t", col.names = TRUE, quote = FALSE)

# Compress the BED file with bgzip
cat("Compressing BED file with bgzip...\n")
bgzip_cmd <- paste("bgzip -f", output_bed)
system(bgzip_cmd)

cat("Process completed.\n")
cat("Output file:", paste0(output_bed, ".gz"), "\n")


Writing BED file to: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/3_phenotype_processing/Astro/Astro_kellis_snatac_phenotype.bed 
Compressing BED file with bgzip...
Process completed.
Output file: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/3_phenotype_processing/Astro/Astro_kellis_snatac_phenotype.bed.gz 


### Format B: Regions Peak Filtering
**Input:**
- `{celltype}_filtered_raw_counts.txt` from Step 1 Option B (in `2_residuals/{celltype}/`)

**Process:**
1. Reads filtered raw counts for each cell type
2. Parses peak coordinates from peak IDs (format: `chr-start-end`)
3. Calculates peak metrics:
   - `peakwidth`: End - Start
   - `midpoint`: (Start + End) / 2
4. Filters for **specific genomic regions of interest**:
   - **Chr7:** 28,000,000 - 28,300,000 bp (300kb region)
   - **Chr11:** 85,050,000 - 86,200,000 bp (1.15Mb region)
5. Includes peaks that overlap these regions (start, end, or span the boundaries)
6. Calculates summary statistics:
   - `total_count`: Sum of counts across all samples per peak
   - `weighted_count`: total_count / peakwidth (normalizes for peak size)

**Output:** `output/4_regions/{celltype}/`
- `filtered_regions_of_interest.txt`: Full count data for peaks in target regions (all samples × selected peaks)
- `filtered_regions_of_interest_summary.txt`: Peak metadata with coordinates and count statistics

**Use Case:** 
Hypothesis-driven analysis of specific genomic loci (e.g., AD risk loci like APOE region, TREM2 locus) where biological variation should be preserved for interpretation.

#### Filter and save data for a specific cell type

In [43]:
# Function to filter and save data for a specific cell type with additional summary information
filter_and_save_by_celltype <- function(celltype) {
  # Create output directory
  peak_output_dir <- file.path(output_dir,"3_regions", celltype)
  dir.create(peak_output_dir, recursive = TRUE, showWarnings = FALSE)
  
  # Load filtered raw counts for the cell type
  input_file <- file.path(output_dir,"2_residuals", celltype, paste0(celltype, "_filtered_raw_counts.txt"))
  
  # Check if file exists before reading
  if (!file.exists(input_file)) {
    cat("File not found:", input_file, "\n")
    return(FALSE)
  }
  
  cat("Processing", celltype, "data from:", input_file, "\n")
  
  # Read data - handling the row names issue
  cell_data <- fread(input_file, check.names = TRUE)
  
  # If the first column has no name (it's row names), give it a proper name
  if (names(cell_data)[1] == "V1") {
    setnames(cell_data, "V1", "peak_id")
  }
  
  # Parse coordinates from peak IDs
  cell_data$chr <- gsub("^(chr[^-]+)-.*$", "\\1", cell_data$peak_id)
  cell_data$start <- as.numeric(gsub("^chr[^-]+-([0-9]+)-.*$", "\\1", cell_data$peak_id))
  cell_data$end <- as.numeric(gsub("^chr[^-]+-[0-9]+-([0-9]+)$", "\\1", cell_data$peak_id))
  
  # Calculate additional metrics
  cell_data$peakwidth <- cell_data$end - cell_data$start
  cell_data$midpoint <- (cell_data$start + cell_data$end) / 2
  
  # Filter for chr7 and chr11
  chr_filtered <- cell_data[cell_data$chr %in% c("chr7", "chr11"), ]
  
  # Filter for the specific regions
  region_filtered <- chr_filtered[
    # Chr7: 28,000kb-28,300kb
    (chr_filtered$chr == "chr7" & 
     ((chr_filtered$start >= 28000000 & chr_filtered$start <= 28300000) | 
      (chr_filtered$end >= 28000000 & chr_filtered$end <= 28300000) |
      (chr_filtered$start <= 28000000 & chr_filtered$end >= 28300000))) |
    # Chr11: 85,050kb-86,200kb
    (chr_filtered$chr == "chr11" & 
     ((chr_filtered$start >= 85050000 & chr_filtered$start <= 86200000) | 
      (chr_filtered$end >= 85050000 & chr_filtered$end <= 86200000) |
      (chr_filtered$start <= 85050000 & chr_filtered$end >= 86200000))),
  ]
  
  # Report results
  cat("Found", nrow(region_filtered), "regions of interest for", celltype, "\n")
  
  # Save the original filtered data (with all columns)
  output_file <- file.path(peak_output_dir, paste0(celltype,"_filtered_regions_of_interest.txt"))
  write.table(region_filtered, output_file, sep="\t", quote=FALSE, row.names=FALSE)
  cat("Saved filtered data to:", output_file, "\n")
  
  # Calculate total count for each peak (sum across all samples)
  # Get only the numeric columns (exclude the metadata columns we added)
  meta_cols <- c("peak_id", "chr", "start", "end", "peakwidth", "midpoint")
  count_cols <- setdiff(names(region_filtered), meta_cols)
  
  # Ensure all count columns are numeric
  region_filtered_counts <- region_filtered[, ..count_cols]
  region_filtered_counts <- as.data.frame(apply(region_filtered_counts, 2, as.numeric))
  
  # Calculate total count
  region_filtered$total_count <- rowSums(region_filtered_counts)
  
  # Calculate weighted count (total count / peakwidth)
  region_filtered$weighted_count <- region_filtered$total_count / region_filtered$peakwidth
  
  # Create a summary data frame with just the metadata columns
  summary_df <- data.table(
    peak_id = region_filtered$peak_id,
    chr = region_filtered$chr,
    start = region_filtered$start,
    end = region_filtered$end,
    midpoint = region_filtered$midpoint,
    peakwidth = region_filtered$peakwidth,
    total_count = region_filtered$total_count,
    weighted_count = region_filtered$weighted_count
  )
  
  # Save the summary data
  summary_file <- file.path(peak_output_dir, paste0(celltype,"_filtered_regions_of_interest_summary.txt"))
  write.table(summary_df, summary_file, sep="\t", quote=FALSE, row.names=FALSE)
  cat("Saved summary data to:", summary_file, "\n\n")
  
  return(TRUE)
}

# List of cell types to process
celltypes <- c("Mic", "Astro", "Oligo", "Exc", "Inh", "OPC")


# Process each cell type
for (ct in celltypes) {
  filter_and_save_by_celltype(ct)
}

File not found: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/Mic/Mic_filtered_raw_counts.txt 
File not found: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/Astro/Astro_filtered_raw_counts.txt 
File not found: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/Oligo/Oligo_filtered_raw_counts.txt 
Processing Exc data from: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/Exc/Exc_filtered_raw_counts.txt 


Warning message in fread(input_file, check.names = TRUE):
“Detected 83 column names but the data has 84 columns (i.e. invalid file). Added an extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


Found 276 regions of interest for Exc 
Saved filtered data to: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/3_regions/Exc/Exc_filtered_regions_of_interest.txt 
Saved summary data to: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/3_regions/Exc/Exc_filtered_regions_of_interest_summary.txt 

File not found: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/Inh/Inh_filtered_raw_counts.txt 
File not found: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals/OPC/OPC_filtered_raw_counts.txt 


## Alternative Pseudobulk Pipeline with Batch Correction

This is an alternative preprocessing approach using ComBat-seq for explicit batch correction. It is from a different dataset (multiome) but demonstrates an alternative strategy when batch effects are severe.

---

#### When to Use This Approach:
- Strong batch effects that need active correction (not just covariate adjustment)
- Data from multiple sequencing runs with substantial technical artifacts
- When batch confounds with biological variables of interest
- Visible batch clusters in PCA/t-SNE plots

---

**Input:**
- QC'd Seurat object with metadata: `{celltype}_qced.rds`
- Pseudobulk peak counts: `{celltype}.rds`
- Sample covariates: `rosmap_cov.txt`
- Batch information: `SampleSheet.csv` and `sampleSheetAfterQc.csv`
- hg38 blacklist: `hg38-blacklist.v2.bed.gz`

**Process:**
1. Loads Seurat object and extracts metadata
2. Loads pseudobulk peak count matrix
3. Calculates technical QC metrics per sample:
   - `TSSEnrichment`: Median TSS enrichment
   - `NucleosomeRatio`: Median nucleosome ratio
   - `LogPercMt`: Log-transformed percent mitochondrial reads
   - `LogUniqueFrags`: Log-transformed unique fragments per sample
4. Filters blacklisted genomic regions using `foverlaps()`
5. Calculates peak metrics:
   - `LogTotalUniquePeaks`: Log-transformed count of unique peaks detected
6. Merges with demographic covariates (msex, age_death, pmi, study)
7. Creates DGEList object
8. Applies expression filtering with `filterByExpr()`:
   - `min.count = 5`: Minimum 5 reads in at least one sample
   - `min.total.count = 7`: Minimum 7 total reads across all samples
   - `min.prop = 0.7`: Peak must be expressed in ≥70% of samples
9. TMM normalization with `calcNormFactors()`
10. **Batch processing:**
    - Loads sequencing batch information from sample sheets
    - Filters singleton batches (batches with only 1 sample)
    - Filters samples with low library sizes (< 5000 recommended)
11. **ComBat-seq batch correction:**
    ```r
    adjusted_counts <- ComBat_seq(
      counts = dge$counts, 
      batch = batches
    )
    ```
12. Fits linear model on batch-corrected counts using `voom()` and `lmFit()`:
    ```r
    model <- ~ pmi + msex + age_death + 
               TSSEnrichment + NucleosomeRatio + LogPercMt +
               LogUniqueFrags + LogTotalUniquePeaks + 
               study
    ```
    Note: Batch is NOT in the model because it was corrected by ComBat-seq
13. Calculates residuals using `predictOffset()`: `offset + residuals`
    - `offset`: Predicted expression at median/reference covariate values
    - `residuals`: Unexplained variation after removing covariate effects

However, ComBat-seq encountered persistent errors with this dataset:
```
Error in .compressOffsets(y, lib.size = lib.size, offset = offset):
offsets must be finite values
```

**Issues with ComBat-seq for this data:**
- Dataset had 232 samples across 60 batches (many small batches)
- Error persisted even after:
  - Filtering samples with low library sizes (< 5000)
  - Removing singleton batches
  - Ensuring all counts and library sizes were finite
  - Verifying no zero-sum peaks
- Likely due to internal ComBat-seq edge case with highly fragmented batch structure

**Solution:** Use limma's `removeBatchEffect` which operates on log-CPM values and is more robust to small batch sizes.

**Process:**
1. Loads Seurat object and extracts metadata
2. Loads pseudobulk peak count matrix
3. Calculates technical QC metrics per sample:
   - `TSSEnrichment`: Median TSS enrichment
   - `NucleosomeRatio`: Median nucleosome ratio
   - `LogPercMt`: Log-transformed percent mitochondrial reads
   - `LogUniqueFrags`: Log-transformed unique fragments per sample
4. Filters blacklisted genomic regions using `foverlaps()`
5. Calculates peak metrics:
   - `LogTotalUniquePeaks`: Log-transformed count of unique peaks detected
6. Merges with demographic covariates (msex, age_death, pmi, study)
7. Creates DGEList object
8. Applies expression filtering with `filterByExpr()`:
   - `min.count = 5`: Minimum 5 reads in at least one sample
   - `min.total.count = 7`: Minimum 7 total reads across all samples
   - `min.prop = 0.7`: Peak must be expressed in ≥70% of samples
9. TMM normalization with `calcNormFactors()`
10. **Batch processing:**
    - Loads sequencing batch information from sample sheets
    - Filters singleton batches (batches with only 1 sample)
11. **Batch correction using limma's removeBatchEffect:**
    ```r
    # Get log-CPM values
    logCPM <- cpm(dge, log=TRUE, prior.count=1)
    
    # Remove batch effects
    adjusted_logCPM <- removeBatchEffect(
      logCPM,
      batch = batches,
      design = model.matrix(~1, data=dge$samples)
    )
    
    # Convert back to counts scale (approximate)
    adjusted_counts <- 2^adjusted_logCPM * mean(dge$$samples$$lib.size) / 1e6
    adjusted_counts <- round(adjusted_counts)
    adjusted_counts[adjusted_counts < 0] <- 0
    ```
12. Updates sample alignment:
    - Ensures valid_samples match current filtered data
    - Aligns covariates with sample order
    - Converts tibble to data.frame and sets rownames
13. Fits linear model on batch-corrected counts using `voom()` and `lmFit()`:
    ```r
    model <- ~ pmi + msex + age_death + TSSEnrichment + NucleosomeRatio + LogPercMt + LogUniqueFrags + LogTotalUniquePeaks + study
    ```
    Note: Batch is NOT in the model because it was corrected by removeBatchEffect
14. Creates new DGEList with batch-corrected counts
15. Recalculates library sizes and TMM normalization factors
16. Calculates residuals using `predictOffset()`: `offset + residuals`
    - `offset`: Predicted expression at median/reference covariate values
    - `residuals`: Unexplained variation after removing covariate effects


**Output:** `output/3_calculateResiduals/{celltype})`
- `{celltype}_results.rds`: Complete results object containing:
  - `dge`: Batch-corrected DGEList
  - `offset`: Predicted offset values
  - `residuals`: Model residuals
  - `batch_adjusted_counts`: removeBatchEffect corrected counts
  - `final_data`: Final adjusted expression (offset + residuals)
  - `valid_samples`: Sample IDs after filtering
  - `design`: Design matrix
  - `fit`: Linear model fit object
- `{celltype}_residuals.txt`: Final covariate-adjusted peak accessibility (log2-CPM scale)


**Key Differences from ComBat-seq:**
- Operates on log-CPM values (not integer counts)
- More robust to small/unbalanced batch sizes
- Does not model mean-variance relationship (simpler correction)
- Approximate back-transformation to count scale


#### Load libraries

In [46]:
library(data.table)
library(stringr)
library(Seurat)
library(dplyr)
library(sva)
library(edgeR)
library(limma)

#### Create output directory

In [47]:
cat("Processing celltype:", celltype, "\n")

residual_out_dir <- file.path(output_dir,"2_residuals_batch_corrected", celltype)
dir.create(residual_out_dir, recursive = TRUE, showWarnings = FALSE)
residual_out_dir

Processing celltype: Astro 


[1] "/restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals_batch_corrected/Astro"

#### Create predictOffset function 

In [48]:
predictOffset <- function(fit) {
  # Define which variables are factors and which are continuous
  usedFactors <- c("study") 
  usedContinuous <- c("pmi", "msex", "age_death", 
                      "TSSEnrichment", "NucleosomeRatio", "LogPercMt",
                      "LogUniqueFrags", "LogTotalUniquePeaks")
  
  # Get indices for factor and continuous variables
  facInd <- unlist(lapply(as.list(usedFactors), 
                         function(f) {return(grep(paste("^", f, sep=""), 
                                                colnames(fit$design)))}))
  contInd <- unlist(lapply(as.list(usedContinuous), 
                          function(f) {return(grep(paste("^", f, sep=""), 
                                                 colnames(fit$design)))}))
  
  # Verify design matrix structure
  stopifnot(!any(duplicated(c(1, facInd, contInd))))
  stopifnot(all(c(1, facInd, contInd) %in% 1:ncol(fit$design)))
  stopifnot(1:ncol(fit$design) %in% c(1, facInd, contInd))
  
  # Create new design matrix with median values
  D <- fit$design
  D[, facInd] <- 0
  medContVals <- apply(D[, contInd], 2, median)
  for (i in 1:length(medContVals)) {
    D[, names(medContVals)[i]] <- medContVals[i]
  }
  
  # Calculate offsets
  stopifnot(all(colnames(coefficients(fit)) == colnames(D)))
  offsets <- apply(coefficients(fit), 1, function(c) {
    return(D %*% c)
  })
  offsets <- t(offsets)
  colnames(offsets) <- rownames(fit$design)
  
  return(offsets)
}

#### Load input data

In [49]:
meta_data = readRDS(file.path(input_dir,"Endothelial_qced.rds"))
meta = meta_data@meta.data
peak <- readRDS(file.path(input_dir,'Endothelial.rds'))

#### Process technical variables

In [50]:
tech_vars <- meta %>%
  group_by(demuxlet_SNG.BEST.GUESS) %>%
  summarise(
    TSSEnrichment = median(TSSEnrichment),
    NucleosomeRatio = median(NucleosomeRatio),
    PercMt = median(percent.mt),
    UniqueFrags = n_distinct(demuxlet_BARCODE)
  ) %>%
  mutate(
    LogPercMt = log(PercMt + 1e-6),
    LogUniqueFrags = log(UniqueFrags + 1e-6)
  )
head(tech_vars)

demuxlet_SNG.BEST.GUESS,TSSEnrichment,NucleosomeRatio,PercMt,UniqueFrags,LogPercMt,LogUniqueFrags
<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
MAP26637867,7.315,1.4334204,0.5755396,87,-0.5524456,4.465908
MAP50106992,6.237,1.5703523,1.7753647,19,0.5740064,2.944439
MAP61344957,14.587,0.7494390,0.2781486,7,-1.2795960,1.945910
ROS11430815,6.606,1.4644619,0.2029770,9,-1.5946577,2.197225
ROS15738428,12.620,0.9908817,0.1889933,32,-1.6660383,3.465736
ROS20945666,7.609,1.6842417,0.3885004,65,-0.9454585,4.174387


#### Process peaks 

In [51]:
# Load blacklist
blacklist_df <- fread(file.path(input_dir,"hg38-blacklist.v2.bed.gz"))
colnames(blacklist_df) <- c("chr", "start", "end", "label")

# Process peak coordinates
peak_df <- data.table(
  peak_name = rownames(peak),
  chr = str_extract(rownames(peak), "chr[0-9XY]+"),
  start = as.integer(str_extract(rownames(peak), "(?<=:)[0-9]+")),
  end = as.integer(str_extract(rownames(peak), "(?<=-)[0-9]+"))
)

# Filter blacklisted peaks
setkey(blacklist_df, chr, start, end)
setkey(peak_df, chr, start, end)
overlapping_peaks <- foverlaps(peak_df, blacklist_df, nomatch=0)
blacklisted_peaks <- unique(overlapping_peaks$peak_name)
filtered_peak <- peak[!rownames(peak) %in% blacklisted_peaks,]

# Calculate peak metrics
peak_metrics <- data.frame(
  sample = colnames(filtered_peak),
  TotalUniquePeaks = colSums(filtered_peak > 0)
) %>%
  mutate(LogTotalUniquePeaks = log(TotalUniquePeaks))

#### Load and merge covariates

In [52]:
covariates <- fread(file.path(input_dir,'rosmap_cov.txt')) %>%
  select('#id', msex, age_death, pmi, study)

all_covs <- tech_vars %>%
  inner_join(peak_metrics, by = c("demuxlet_SNG.BEST.GUESS" = "sample")) %>%
  inner_join(covariates, by = c("demuxlet_SNG.BEST.GUESS" = "#id"))

cat("Number of samples after joining:", nrow(all_covs), "\n")
cat("Sample IDs:", paste(head(all_covs$demuxlet_SNG.BEST.GUESS), collapse=", "), "...\n")

# Impute missing values
for(col in c("pmi", "age_death")) {
  all_covs[[col]][is.na(all_covs[[col]])] <- median(all_covs[[col]], na.rm=TRUE)
}

Number of samples after joining: 233 
Sample IDs: MAP26637867, MAP50106992, MAP61344957, ROS11430815, ROS15738428, ROS20945666 ...


#### Create DGE object

In [53]:
dge <- DGEList(
  counts = filtered_peak,
  samples = all_covs
)

#### Filter low counts and normalize

In [54]:
cat("Number of peaks before filtering:", nrow(dge), "\n")

# keep <- filterByExpr(dge) #only 2 peaks left in mic
# default paramter:
# keep <- filterByExpr(y, 
#                      min.count = 10,     # for one sample, min reads 
#                      min.total.count = 15, # min reads overall
#                      min.prop = 0.7) 

keep <- filterByExpr(dge, 
                     min.count = 5,     # for one sample, min reads 
                     min.total.count = 15, # min reads overall
                     min.prop = 0.1,
                     group = NULL) 

dge <- dge[keep, , keep.lib.sizes=TRUE] #mic: from 130930 to 2
cat("Number of peaks after filtering:", nrow(dge), "\n")
dge <- calcNormFactors(dge, method="TMM")

Number of peaks before filtering: 130930 


Warning message in filterByExpr.DGEList(dge, min.count = 5, min.total.count = 15, :
“All samples appear to belong to the same group.”


Number of peaks after filtering: 21197 


#### Load batch information

In [55]:
sample_file <- file.path(input_dir,"SampleSheet.csv")
wgs_qc_file <- file.path(input_dir,"sampleSheetAfterQc.csv")

sample <- fread(sample_file, colClasses = "character")
wgs_qc <- fread(wgs_qc_file, colClasses = "character")
sample <- sample %>%
  inner_join(wgs_qc) %>%
  select(SequencingID, SampleID)

# Extract batch information
batches <- sample$SequencingID
names(batches) <- sample$SampleID

valid_samples <- colnames(dge$counts)
batches <- batches[valid_samples]

Joining with `by = join_by(ProjID)`


#### Run ComBat-seq

In [42]:
# Filter batches with only one sample
batch_counts <- table(batches)
valid_batches <- names(batch_counts[batch_counts > 1])
batches <- batches[batches %in% valid_batches]
valid_samples <- names(batches)

keep <- colnames(dge$counts) %in% names(batches)
dge <- dge[keep, , keep.lib.sizes=TRUE]
batches <- batches[colnames(dge$counts)]
stopifnot(all(colnames(dge$counts) == names(batches)))

cat("Number of samples after batch filtering:", length(valid_samples), "\n")
cat("Number of batches:", length(unique(batches)), "\n")

# Run ComBat-seq
adjusted_counts <- ComBat_seq(
  counts = dge$counts, 
  batch = batches
)

ERROR: Error: all(colnames(dge$counts) == names(batches)) is not TRUE


#### Create model and run voom

In [44]:
model <- ~ pmi + msex + age_death + 
  TSSEnrichment + NucleosomeRatio + LogPercMt +
  LogUniqueFrags + LogTotalUniquePeaks + 
  study

# Update design matrix for remaining samples
design <- model.matrix(model, data=all_covs[valid_samples,])
stopifnot(is.fullrank(design))

dge_adjusted <- dge[, valid_samples]  
dge_adjusted$counts <- adjusted_counts[, valid_samples] 

# Run voom and fit model
v <- voom(dge_adjusted[, valid_samples], design, plot=FALSE)
fit <- lmFit(v, design)
fit <- eBayes(fit)

# Calculate offset and residuals
offset <- predictOffset(fit)
resids <- residuals(fit, y=v)

# Verify dimensions
stopifnot(all(rownames(offset) == rownames(resids)) &
          all(colnames(offset) == colnames(resids)))

# Final adjusted data
stopifnot(all(dim(offset) == dim(resids)))
stopifnot(all(colnames(offset) == colnames(resids)))

final_data <- offset + resids

ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


#### Run LIMMA as Combat-seq alternative 

In [56]:
# Alternative: Use limma's removeBatchEffect instead
# Get log-CPM values
logCPM <- cpm(dge, log=TRUE, prior.count=1)

# Remove batch effects
adjusted_logCPM <- removeBatchEffect(
  logCPM,
  batch = batches,
  design = model.matrix(~1, data=dge$samples)
)

# Convert back to counts scale (approximate)
adjusted_counts <- 2^adjusted_logCPM * mean(dge$samples$lib.size) / 1e6
adjusted_counts <- round(adjusted_counts)
adjusted_counts[adjusted_counts < 0] <- 0

#### Create model and run voom

In [57]:
# Update valid_samples to match current data
valid_samples <- colnames(dge)

# Get aligned covariates
filtered_covs <- all_covs[match(valid_samples, all_covs$demuxlet_SNG.BEST.GUESS), ]
filtered_covs <- as.data.frame(filtered_covs)  # Convert from tibble
rownames(filtered_covs) <- valid_samples


# Build model formula
model_formula <- ~ pmi + msex + age_death + 
  TSSEnrichment + NucleosomeRatio + LogPercMt +
  LogUniqueFrags + LogTotalUniquePeaks + 
  study

# Create design matrix
design <- model.matrix(model_formula, data=filtered_covs)
rownames(design) <- valid_samples

stopifnot(is.fullrank(design))
stopifnot(all(rownames(design) == colnames(dge)))

# Create properly formatted DGEList with adjusted counts
dge_adjusted <- DGEList(
  counts = adjusted_counts,
  samples = filtered_covs
)

# Recalculate library sizes and normalization factors
dge_adjusted$samples$lib.size <- colSums(dge_adjusted$counts)
dge_adjusted <- calcNormFactors(dge_adjusted, method="TMM")

stopifnot(all(rownames(design) == colnames(dge_adjusted)))

# Run voom and fit model
v <- voom(dge_adjusted, design, plot=FALSE)
fit <- lmFit(v, design)
fit <- eBayes(fit)

# Calculate offset and residuals
offset <- predictOffset(fit)
resids <- residuals(fit, y=v)

# Final adjusted data
final_data <- offset + resids


#### Save results

In [58]:
saveRDS(list(
  dge = dge_adjusted,
  offset = offset,
  residuals = resids,
  batch_adjusted_counts = adjusted_counts,
  final_data = final_data,
  valid_samples = valid_samples,
  design = design,
  fit = fit
), file = file.path(residual_out_dir, paste0(celltype, "_results.rds")))

# Write final residual data to file
write.table(final_data,
            file = file.path(residual_out_dir, paste0(celltype, "_residuals.txt")), 
            quote=FALSE)

cat("Results saved to:", residual_out_dir, "\n")               

Results saved to: /restricted/projectnb/xqtl/jaempawi/atac_seq/output/kellis/2_residuals_batch_corrected/Astro 
